In [121]:
import ROOT
import ROOT as rt

In [123]:
year = 2018
# njet = 0
for njet in [0,1,2]:
    file = ROOT.TFile(f"{year}_njet{njet}.root", "READ")
    save_path = "./plots"
    workspace = file.Get("zpt_Workspace")
    # target_nbins = 50
    # for target_nbins in [25, 50, 100, 250]:
    print(f"{year} njet{njet}------------------------------------------------------------------------------------------------------")
    for target_nbins in [50, 100]:
        hist_data = workspace.obj("hist_data").Clone("hist_data_clone")
        hist_dy = workspace.obj("hist_dy").Clone("hist_dy_clone")
        orig_nbins = hist_data.GetNbinsX()
        rebin_coeff = int(orig_nbins/target_nbins)
        print(f"rebin_coeff: {rebin_coeff}")
        hist_data = hist_data.Rebin(rebin_coeff, "rebinned hist_data") 
        hist_dy = hist_dy.Rebin(rebin_coeff, "rebinned hist_dy") 
        
        hist_SF = hist_data.Clone("hist_SF")
        hist_SF.Divide(hist_dy)
        
        
        
        # Draw the histogram and fit
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins Data and DY", 800, 600)
        hist_data.SetLineColor(ROOT.kRed)
        hist_dy.SetLineColor(ROOT.kBlue)
        # Change the plot title
        hist_data.SetTitle(f"njet {njet} {target_nbins} bins Data and DY")
        hist_data.Draw()
        
        hist_dy.Draw("SAME")
        # Add a legend
        legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
        legend.AddEntry(hist_data, "Data", "l")  # "l" means line style
        legend.AddEntry(hist_dy, "DY", "l")
        legend.Draw()
        canvas.SetLogy(1)
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_DataDy_Hist.pdf")
        
        canvas = ROOT.TCanvas("canvas", f"{target_nbins} bins SF hist", 800, 600)
        hist_SF.SetTitle(f"njet {njet} {target_nbins} bins SF")
        hist_SF.SetMinimum(0.5)  # Set the lower bound of the Y-axis
        hist_SF.SetMaximum(4)  # Set the upper bound of the Y-axis
        hist_SF.Draw()
        
        canvas.Update()
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.png")
        canvas.SaveAs(f"{save_path}/{year}_njet{njet}_{target_nbins}Bins_SF_Hist.pdf")

        
        dimuon_pt = ROOT.RooRealVar("dimuon_pt", "Dimuon pT", 0, 200)

        # Convert the TH1F histogram to a RooDataHist
        roo_his_SF = ROOT.RooDataHist("roo_hist", "RooFit Histogram", ROOT.RooArgList(dimuon_pt), hist_SF)
        
        # Print information about the RooDataHist
        roo_his_SF.Print()
        


        for order in range(3,13):
            # Define two polynomial orders
            order_low = order
            order_high = order + 1

            coeff_list_low = [rt.RooRealVar(f"a{ix}_{order_low}_low", f"a{ix}_{order_low}_low", 0, -5, 5) for ix in range(order_low+1)]
            name = f"{order_low}_poly_low"
            poly_low = rt.RooChebychev(name, name, dimuon_pt,  
                                      coeff_list_low)
            _ = poly_low.fitTo(roo_his_SF, Save=True)
            fit_results = poly_low.fitTo(roo_his_SF, Save=True)
            fit_results.Print()
            raise ValueError
            
    
            # # Fit with the lower-order polynomial
            # polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order_low + 1)])
            # polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # # Define the TF1 function with the generated expression
            # fit_func_low = polynomial_func
            # _ = hist_SF.Fit(fit_func_low, "LS")
            # fit_low = hist_SF.Fit(fit_func_low, "LS")
            # # print(fit_low.minNll())
            # print(fit_func_low.GetLogLikelihood())
            # raise ValueError
            
            # chi2_low = fit_func_low.GetChisquare()
            # ndf_low = fit_func_low.GetNDF()
            # # log_likelihood_low = fit_func_low.GetLogLikelihood()
            # # print(f"log_likelihood_low: {log_likelihood_low}")
            
            # # Fit with the higher-order polynomial
            # polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order_high + 1)])
            # # Define the TF1 function with the generated expression
            # polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)
            # # Define the TF1 function with the generated expression
            # fit_func_high = polynomial_func
            # _ = hist_SF.Fit(fit_func_high, "S")
            # fit_high = hist_SF.Fit(fit_func_high, "S")
            
            # chi2_high = fit_func_high.GetChisquare()
            # ndf_high = fit_func_high.GetNDF()
            
            # # Calculate F-statistic
            # delta_chi2 = chi2_low - chi2_high
            # delta_dof = ndf_high - ndf_low
            # f_statistic = (delta_chi2 / delta_dof) / (chi2_high / ndf_high)

            # print(f"f_statistic: {f_statistic}")
            # # Calculate the p-value (use scipy.stats.f for F-distribution)
            # # p_value = 1 - f.cdf(f_statistic, delta_dof, ndf_high)
            # p_value = 1 - f.cdf(f_statistic, ndf_low, ndf_high)
            
            # # Print results
            # print(f"Lower-order {target_nbins} bins polynomial (pol{order_low}): chi2 = {chi2_low}, ndf = {ndf_low}")
            # print(f"Higher-order {target_nbins} bins polynomial (pol{order_high}): chi2 = {chi2_high}, ndf = {ndf_high}")
            # print(f"F-statistic {target_nbins} bins: {f_statistic}")
            # print(f"P-value {target_nbins} bins: {p_value}")
            
            # if p_value < 0.05:  # Typically, p-value < 0.05 indicates significant improvement
            #     print(f"Higher-order {order_high} polynomial significantly improves the fit.")
            # else:
            #     print(f"Higher-order {order_high} polynomial does not significantly improve the fit.")

2018 njet0------------------------------------------------------------------------------------------------------
rebin_coeff: 10


ValueError: 

RooDataHist::roo_hist[dimuon_pt] = 50 bins (59.5742 weights)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(3_poly_low_over_3_poly_low_Int[dimuon_pt]) fixing normalization set for coefficient determination to observables in data
[#0] WARNING:InputArguments -- RooAbsPdf::fitTo(3_poly_low): WARNING: a likelihood fit is requested of what appears to be weighted data.
       While the estimated values of the parameters will always be calculated taking the weights into account,
       there are multiple ways to estimate the errors of the parameters. You are advised to make an
       explicit choice for the error calculation:
           - Either provide SumW2Error(true), to calculate a sum-of-weights-corrected HESSE error matrix
             (error will be proportional to the number of events in MC).
           - Or provide SumW2Error(false), to return errors from original HESSE error matrix
             (which will be proportional to the sum of the weights, i.e., a dataset with <sum of weights> event

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50Bins_DataDy_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50Bins_DataDy_Hist.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file ./plots/2018_njet0_50Bins_SF_Hist.png has been created
Info in <TCanvas::Print>: pdf file ./plots/2018_njet0_50Bins_SF_Hist.pdf has been created
Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       315.6429406 Edm =      0.2024577048 NCalls =     17
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 315.6429406
  Edm           : 0.2024577048
  Internal parameters:	[                0                0                0                0]	
  Internal gradient  :	[     -5.256628619      6.499928431     -11.21614805   

In [69]:
# Draw the histogram and fit

order = 7

canvas = ROOT.TCanvas("canvas", f"{order}th-Order Polynomial Fit", 800, 600)


# Define the order of the polynomial

# fit_func = hist_SF.GetFunction(f"pol{order}")

# Generate the polynomial expression dynamically
polynomial_expr = " + ".join([f"[{i}]*x**{i}" for i in range(order + 1)])

# Define the TF1 function with the generated expression
polynomial_func = ROOT.TF1(f"poly{order}", polynomial_expr, -5, 5)

# Set initial parameter guesses (optional)
for i in range(order + 1):
    polynomial_func.SetParameter(i, 1)  # Initial guess for each parameter

fit_func = polynomial_func
# print(fit_func)

_ = hist_SF.Fit(fit_func, "S")  # "S" option ensures we get the fit result
fit_result = hist_SF.Fit(fit_func, "S")  # "S" option ensures we get the fit result
# print(fit_result)
# fit_result.Print()
# Extract chi2 and dof
# fit_func = hist_SF.GetFunction("pol3")
chi2 = fit_func.GetChisquare()  # Total chi-squared
ndf = fit_func.GetNDF()  # Number of degrees of freedom

# Calculate chi2/dof
chi2_dof = chi2 / ndf if ndf > 0 else float("inf")

# Print fit results
print(f"Chi2: {chi2}")
print(f"NDF: {ndf}")
print(f"Chi2/NDF: {chi2_dof}")

# Extract parameters and their uncertainties
num_params = fit_func.GetNpar()  # Number of parameters in the fit
print("Fitted parameters and uncertainties:")
for i in range(num_params):
    param_value = fit_func.GetParameter(i)  # Fitted parameter value
    param_error = fit_func.GetParError(i)  # Fitted parameter uncertainty
    print(f"  Parameter {i}: {param_value:.4f} ± {param_error:.4f}")

hist_SF.SetLineColor(ROOT.kBlue)
hist_SF.Draw()
fit_func.SetLineColor(ROOT.kRed)  # Change color for each fit
fit_func.Draw("SAME")  # Draw the fit function on the same canvas


# Add a legend
legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)  # Legend coordinates (x1, y1, x2, y2)
legend.AddEntry(hist_SF, "hist_SF", "l")  # "l" means line style
legend.AddEntry(fit_func, "fit", "l")
legend.Draw()

# Update the canvas
canvas.Update()


# Save the canvas
canvas.SaveAs(f"{save_path}/{year}_njet{njet}_SF_fit.png")


Chi2: 1170.9752966120684
NDF: 42
Chi2/NDF: 27.880364205049247
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1170.98
NDf                       =           42
Edm                       =  1.93314e-09
NCalls                    =          741
p0                        =     0.635804   +/-   0.000745401 
p1                        =    0.0844285   +/-   0.000132114 
p2                        =  -0.00548066   +/-   5.98055e-06 
p3                        =  0.000157687   +/-   1.00132e-07 
p4                        = -2.20465e-06   +/-   7.8029e-10  
p5                        =  1.57946e-08   +/-   5.08576e-12 
p6                        = -5.59425e-11   +/-   2.9218e-14  
p7                        =  7.77105e-14   +/-   1.20302e-16 


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: png file test_polynomial_fits.png has been created


In [112]:
import ROOT
import math

# Create a histogram
hist = ROOT.TH1D("hist", "Example Histogram", 50, -5, 5)

# Fill the histogram with Gaussian-distributed random data
for _ in range(1000):
    hist.Fill(ROOT.gRandom.Gaus(0, 1))

# Fit the histogram with a Gaussian function
fit_result = hist.Fit("gaus", "S")  # "S" option ensures the fit result is accessible
fit_func = hist.GetFunction("gaus")  # Get the fitted function

if fit_func:
    # Extract the log-likelihood value
    log_likelihood = fit_func.GetLogLikelihood()

    # Calculate the likelihood
    likelihood = math.exp(log_likelihood)

    # Print the results
    print(f"Log-likelihood value: {log_likelihood}")
    print(f"Likelihood value: {likelihood}")
else:
    print("Fit function not available.")


AttributeError: 'TF1' object has no attribute 'GetLogLikelihood'

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      34.7164
NDf                       =           29
Edm                       =  9.72069e-09
NCalls                    =           59
Constant                  =      74.9237   +/-   3.00474     
Mean                      =    0.0111669   +/-   0.0336562   
Sigma                     =      1.03102   +/-   0.0253614    	 (limited)
